In [5]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline
import pandas_datareader.data as web


In [2]:
from dotenv import load_dotenv
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2') 

In [3]:
assets = api.list_assets()
tradeable = [asset for asset in assets if asset.tradable]

In [4]:
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])

In [6]:
main_df = pd.DataFrame()
screened_stocks_df = pd.read_csv('../Data/REIT_stocks.csv')
screened_stocks_tickers = screened_stocks_df['Ticker']
tickers= screened_stocks_tickers.tolist()
tickers.pop(2)

'AIII'

In [7]:
timeframe = '1D'
end_date = datetime.now()
start_date = end_date + timedelta(-365)

for count, ticker in enumerate(tickers):
    df = api.get_barset(
        ticker,
        timeframe,
        limit=None,
        start=start_date,
        end=end_date,
        after=None,
        until=None,
    ).df
    df.index = df.index.date
    series = df[ticker]['close']
    series.rename(ticker, inplace =True)
    if main_df.empty:
        main_df = series
    else:
        main_df = pd.concat([main_df,series], axis = 'columns',join='inner')

        
main_df.head()

,UDR,UMH
2019-12-09,47.84,15.75
2019-12-10,47.73,15.80
2019-12-11,46.80,16.20
2019-12-12,46.38,15.91
2019-12-13,46.25,15.96


# Finding S&P 500 
## finding the close prices of sp500 with pandas DataReader

In [8]:
#THIS is the only thing that worked for me to get closing stock information
sp500 = web.DataReader(['sp500'], 'fred', start_date, end_date)
sp500.head()

,sp500
DATE,
2019-05-06,2932.47
2019-05-07,2884.05
2019-05-08,2879.42
2019-05-09,2870.72
2019-05-10,2881.40


In [9]:
read_stocks = web.DataReader(tickers, 'yahoo',start_date,end_date)
close_stocks = read_stocks['Close']
close_stocks.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2019-05-06,47.110001,16.42,49.759998,24.160000,16.10,200.570007,29.010000,11.40,14.19,100.889999,...,39.110001,28.030001,10.07,12.84,6.11,42.150002,123.919998,5.00,44.369999,13.65
2019-05-07,46.250000,16.16,49.080002,23.660000,15.92,196.789993,29.010000,11.11,14.18,99.070000,...,38.060001,26.889999,10.03,12.45,6.01,41.509998,121.839996,4.85,43.549999,13.15
2019-05-08,45.910000,16.10,49.110001,23.530001,15.93,196.449997,29.010000,11.04,14.22,98.750000,...,39.970001,26.809999,11.08,12.52,6.12,41.540001,121.150002,4.92,43.599998,13.22
2019-05-09,45.919998,16.26,49.410000,23.639999,15.88,198.529999,29.010000,10.94,14.17,99.849998,...,41.349998,26.930000,11.00,12.52,6.21,41.720001,122.059998,5.10,43.709999,13.30
2019-05-10,45.910000,16.41,49.889999,23.790001,16.17,200.710007,29.219999,10.99,14.22,100.980003,...,41.310001,26.639999,11.00,12.69,6.32,42.240002,122.959999,4.86,44.400002,13.31


In [10]:
sp500_returns = sp500.pct_change()
sp500_returns.dropna(inplace = True)
sp500_returns.head()

,sp500
DATE,
2019-05-07,-0.016512
2019-05-08,-0.001605
2019-05-09,-0.003021
2019-05-10,0.003720
2019-05-13,-0.024131


In [11]:
stocks_returns = close_stocks.pct_change()
stocks_returns.dropna(inplace =True)
stocks_returns.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-13,0.004533,0.012048,0.015524,0.014792,0.002320,0.012441,0.011032,0.006873,-0.009217,0.012620,...,0.017457,0.005979,-0.019964,0.007607,0.003995,0.006475,0.016620,-0.040000,0.014034,-0.013690
2019-11-14,0.005801,0.007937,0.003356,0.008439,0.029321,0.004868,0.000000,0.000000,0.005233,0.006593,...,0.007626,0.001564,0.002778,0.007550,0.002653,0.013671,0.004008,0.031250,0.007549,-0.014541
2019-11-15,0.005554,0.006187,0.007432,0.008368,0.020240,0.008983,0.057800,0.005119,-0.017930,0.007447,...,0.009249,0.000000,-0.000923,-0.007493,0.025132,0.006942,0.010137,-0.006734,0.008117,-0.004024
2019-11-18,0.008286,-0.010062,0.007008,-0.001886,0.005878,0.003449,0.016448,-0.008489,-0.004711,0.002672,...,0.001666,0.009994,0.001848,0.002745,0.015484,0.005318,0.012733,-0.003390,0.005161,-0.006734
2019-11-19,-0.000421,0.002823,0.001648,-0.002645,-0.005113,0.008315,-0.014262,0.000000,0.015976,0.001776,...,-0.006446,-0.005257,0.000000,0.001369,0.010165,0.002743,0.013688,0.020408,0.003902,0.010169


# Beta Section 

In [12]:
#covariance for the stocks with sp500
def get_covariance(stocks_returns):
    covariance_df = pd.DataFrame()
    for symbol in stocks_returns:
        covariance = stocks_returns[symbol].cov(sp500_returns['sp500'])
        covariance_df = covariance_df.append(covariance)
    return covariance_dict


In [13]:
#variance 
def get_variance():
    variance_dict = {}
    for symbol in stocks_returns:
        variance = stocks_returns[symbol].var()
        variance_dict.update({symbol: variance})
    return variance_dict


In [17]:
#calculating beta
def get_beta(stocks_returns):
    beta_dict = {}
    for symbol in stocks_returns:
        covariance = stocks_returns[symbol].cov(sp500_returns['sp500'])
        variance = stocks_returns[symbol].var()
        beta = covariance/variance
        beta_dict.update({beta:symbol})
    return beta_dict
get_beta(stocks_returns)


{0.41944295300957385: 'ACC',
 0.36560331262178114: 'AHH',
 0.45954754371930717: 'AIV',
 0.6653147068137334: 'AMH',
 0.32002865779151835: 'APTS',
 0.5660986644779483: 'AVB',
 0.2198624576508861: 'BOWFF',
 0.23515703941516333: 'BRG',
 0.3236785816702743: 'BRT',
 0.6291438268700082: 'CPT',
 0.5415653019493749: 'ELS',
 0.6228295884441624: 'EQR',
 0.6017330908138875: 'ESS',
 -0.002085403541380801: 'GADS',
 -0.2491495879533982: 'IIPZF',
 0.5937055846448906: 'INVH',
 0.4644691411800893: 'IRT',
 0.6087256883014851: 'MAA',
 0.3866922611931805: 'MRTI',
 0.38007354437293334: 'NXRT',
 0.45778811838660155: 'OPI',
 0.38823310618187457: 'RESI',
 0.28293520803984346: 'RPT',
 0.26482574343505416: 'SNR',
 0.3553706633745808: 'SRC',
 0.5308242109481697: 'SUI',
 0.11541090327445676: 'UDFI',
 0.6134628611064512: 'UDR',
 0.4378915577925243: 'UMH'}

# Rolling Averages 
## Getting a 20 day and 40 day rolling averages

In [18]:
# Calculating Rolling Average of 20 days
rolling_20 = close_stocks.rolling(window = 20).mean()
rolling_20.dropna(inplace = True)
rolling_20.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-10,47.4405,17.9475,53.3780,26.4150,13.5845,214.059501,35.9795,12.0240,17.6610,110.9470,...,47.941500,32.894500,11.5960,14.5400,7.7870,51.4865,160.580001,3.0550,47.860999,15.2260
2019-12-11,47.4605,18.0080,53.3135,26.4195,13.6250,214.213501,36.0805,12.0475,17.7010,110.8895,...,47.989999,33.003500,11.6755,14.5530,7.8065,51.5840,160.672501,3.0555,47.849999,15.2690
2019-12-12,47.4225,18.0525,53.1815,26.3860,13.6390,214.092001,36.1410,12.0520,17.7275,110.7110,...,47.934499,33.050999,11.7530,14.5505,7.8080,51.5820,160.485501,3.0595,47.784500,15.3075
2019-12-13,47.3825,18.0870,53.0480,26.3525,13.6235,213.893002,36.1795,12.0480,17.7190,110.4005,...,47.833499,32.994999,11.8265,14.5365,7.8090,51.5345,160.251001,3.0610,47.695000,15.3600
2019-12-16,47.3410,18.1200,52.9055,26.3185,13.6065,213.635001,36.1275,12.0465,17.7230,110.0675,...,47.659499,32.941999,11.8955,14.5365,7.8005,51.4855,159.872501,3.0635,47.589000,15.4045


In [19]:
# Calculating Rolling Average of 40 days
rolling_40 = close_stocks.rolling(window = 40).mean()
rolling_40.dropna(inplace = True)
rolling_40.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-09,46.74800,18.13050,52.20050,26.13700,13.38800,211.077251,35.70575,11.93450,17.49475,108.270749,...,46.18950,32.40625,11.99200,14.56275,7.72825,50.27525,155.110751,2.96700,47.02325,15.46925
2020-01-10,46.73350,18.15000,52.16800,26.15075,13.38950,211.063750,35.76075,11.94200,17.49650,108.192500,...,46.17625,32.43150,12.02125,14.55875,7.73125,50.27250,154.991750,2.96950,46.99875,15.47750
2020-01-13,46.71875,18.16900,52.12450,26.16425,13.39700,211.049501,35.80650,11.95025,17.50225,108.099249,...,46.15000,32.47400,12.05875,14.55475,7.73650,50.26875,154.890250,2.97025,46.97125,15.49350
2020-01-14,46.69800,18.18350,52.06275,26.16900,13.39725,210.971751,35.85875,11.95575,17.50650,107.991249,...,46.10925,32.51600,12.09025,14.54950,7.74325,50.26050,154.772750,2.96900,46.93075,15.52125
2020-01-15,46.67475,18.19875,52.01075,26.17225,13.39750,210.906001,35.87300,11.96350,17.52100,107.882749,...,46.07475,32.58100,12.12400,14.54900,7.74525,50.25600,154.624501,2.96800,46.89150,15.55325


In [20]:
rolling_100 = close_stocks.rolling(window = 100).mean()
rolling_100.dropna(inplace =True)

# Top 5 for beta

In [21]:
#getting the top pick of the day
beta_dict = get_beta(stocks_returns)  
search_key = 1
result = beta_dict.get(search_key) or beta_dict[min(beta_dict, key = lambda key: abs(key-search_key))]    
  
print("The stock best to choose today according to beta: " + str(result))


The stock best to choose today according to beta: AMH


In [59]:
#getting the top 5 picks of the day
def top5_beta(beta_dict):
    sorted_beta_list = sorted(beta_dict, key=lambda key: abs(key), reverse = True)
    top_5_beta_list= sorted_beta_list[0:5]
    top_5_beta_dict = {}
    for keys in top_5_beta_list:
        for values in beta_dict.keys():
            if keys == values:
                top_5_beta_dict.update({beta_dict[values]:keys})
    return top_5_beta_dict
top_5_beta(beta_dict)

{'AMH': 0.6653147068137334,
 'CPT': 0.6291438268700082,
 'EQR': 0.6228295884441624,
 'UDR': 0.6134628611064512,
 'MAA': 0.6087256883014851}

# Filtered By Percent Change 
* within a 30 day range Can adust days if need be

In [28]:
avg_pct_change = stocks_returns.tail(30).mean()*100
avg_pct_change_df = pd.Series.to_frame(avg_pct_change)
avg_pct_change_df.columns = ['Percent_Change']
avg_pct_change_df.head()

,Percent_Change
Symbols,
ACC,1.251836
AHH,0.305703
AIV,0.729208
AMH,0.482699
APTS,0.012898


In [29]:
def top5_pct_change(pct_change_df):
    sorted_pct_df=pct_change_df.sort_values(by='Percent_Change', ascending=False)
    top5_df=sorted_pct_df.head()
    return top5_df

top5_pct_change(avg_pct_change_df)

,Percent_Change
Symbols,
GADS,2.920425
SNR,1.997306
BOWFF,1.487719
ACC,1.251836
UMH,0.984026


# Standard Deviation

In [30]:
def get_std(returns):
    std_dev_returns = returns.std()
    std_dev_df = pd.Series.to_frame(std_dev_returns)
    std_dev_df.columns = ['Std_Dev']
    return std_dev_df

get_std(stocks_returns).head()

,Std_Dev
Symbols,
ACC,0.051924
AHH,0.055304
AIV,0.045609
AMH,0.033778
APTS,0.049006


In [130]:
std_dev_df = get_std(stocks_returns)
def top5_std_dev(std_dev_df):
    sorted_std_df=std_dev_df.sort_values(by='Std_Dev', ascending=False)
    top5_df=sorted_std_df.head()
    return top5_df
top5_std_dev(std_dev_df)

,Std_Dev
Symbols,
GADS,0.426558
SNR,0.072081
BRG,0.070423
RPT,0.059798
BOWFF,0.059719


# Filters between Std and beta and Sharp Ratios

In [78]:
def get_sharpe_ratios(returns):
    sharpe_ratios_df = (returns.mean() * 252) / (returns.std() * np.sqrt(252))
    return sharpe_ratios_df
stock_sharpe_ratio=get_sharpe_ratios(stocks_returns)
stock_sharpe_ratio.head()

Symbols
ACC    -0.462703
AHH    -1.033510
AIV    -0.728435
AMH    -0.131196
APTS   -1.244828
dtype: float64

In [72]:
sharpe_ratio =stock_sharpe_ratio
def top5_sharpe(sharpe_ratio):
    sorted_sharpe=sharpe_ratio.sort_values(ascending=False)
    top5_df=sorted_sharpe.head()
    return top5_df
top5_sharpe(sharpe_ratio)

Symbols
GADS    2.372325
RESI    0.250020
ELS    -0.052518
AMH    -0.131196
OPI    -0.183230
dtype: float64

In [134]:
beta = top5_beta(beta_dict)
std = top5_std_dev(std_dev_df)
sharpe = top5_sharpe(sharpe_ratio)
def top5_picks(beta, std, sharpe):
    top5_df = pd.DataFrame()
    bad_picks = []
    for std_stocks in std.index:
        if std_stocks in sharpe.index:
            
    return top5_df
            
top5_picks(beta,std,sharpe)


IndentationError: expected an indented block (<ipython-input-134-613133f7c1dd>, line 11)